# 用前须知

## xtdata提供和MiniQmt的交互接口，本质是和MiniQmt建立连接，由MiniQmt处理行情数据请求，再把结果回传返回到python层。使用的行情服务器以及能获取到的行情数据和MiniQmt是一致的，要检查数据或者切换连接时直接操作MiniQmt即可。

## 对于数据获取接口，使用时需要先确保MiniQmt已有所需要的数据，如果不足可以通过补充数据接口补充，再调用数据获取接口获取。

## 对于订阅接口，直接设置数据回调，数据到来时会由回调返回。订阅接收到的数据一般会保存下来，同种数据不需要再单独补充。


In [1]:
# 从本地python导入xtquant库，如果出现报错则说明安装失败
from xtquant import xtdata
import time

In [2]:
# 设定一个标的列表
code_list = ["000001.SZ"]
# 设定获取数据的周期
period = "1d"

# 下载标的行情数据
if 1:
    ## 为了方便用户进行数据管理，xtquant的大部分历史数据都是以压缩形式存储在本地的
    ## 比如行情数据，需要通过download_history_data下载，财务数据需要通过
    ## 所以在取历史数据之前，我们需要调用数据下载接口，将数据下载到本地
    # 设置开始和结束时间
    start_date = "20230101"
    end_date = "20230201"
    for i in code_list:
        print(f"开始下载{i}的历史数据，时间区间为{start_date}到{end_date}")
        xtdata.download_history_data(i,period=period,start_time=start_date,end_time=end_date,incrementally=True) # 增量下载行情数据（开高低收,等等）到本地

# 读取本地历史行情数据
history_data = xtdata.get_market_data_ex([],code_list,period=period,count=-1)
print(f"获取到{len(history_data)}历史数据，数据如下：")
print(history_data)
print("=" * 20)



开始下载000001.SZ的历史数据，时间区间为20230101到20230201
***** xtdata连接成功 *****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 127.0.0.1:58610
数据路径: D:\gjzqqmt\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息

获取到1历史数据，数据如下：
{'000001.SZ':                    time   open   high    low  close   volume        amount  \
19910107   663177600000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910108   663264000000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910109   663350400000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910110   663436800000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910111   663523200000  65.74  65.74  65.74  65.74        0  0.000000e+00   
...                 ...    ...    ...    ...    ...      ...           ...   
20250421  1745164800000  11.00  11.13  10.97  11.02  1114184  1.230805e+09   
20250422  1745251200000  11.02  11.06  10.98  11.04   831129  9.159677e+08   
20250423  1745337600000  11.04  11.05  10.97  11.01   58

In [8]:
# 如果需要盘中的实时行情，需要向服务器进行订阅后才能获取
# 订阅后，get_market_data函数于get_market_data_ex函数将会自动拼接本地历史行情与服务器实时行情

# 向服务器订阅数据
sub_res = ''
for i in code_list:
    print(f"开始订阅{i}的实时行情")
    sub_res = xtdata.subscribe_quote(i,period=period,count=10) # count: 数量 -1全部/n: 从结束时间向前数n个
    print(f"订阅{i}的实时行情，返回值为{sub_res}")

# 等待订阅完成
time.sleep(1)
print("=" * 20)

# 获取订阅后的行情
kline_data = xtdata.get_market_data_ex([],code_list,period=period)
print(kline_data)

开始订阅000001.SZ的实时行情
订阅000001.SZ的实时行情，返回值为4
{'000001.SZ':                    time   open   high    low  close   volume        amount  \
19910107   663177600000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910108   663264000000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910109   663350400000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910110   663436800000  65.74  65.74  65.74  65.74        0  0.000000e+00   
19910111   663523200000  65.74  65.74  65.74  65.74        0  0.000000e+00   
...                 ...    ...    ...    ...    ...      ...           ...   
20250421  1745164800000  11.00  11.13  10.97  11.02  1114184  1.230805e+09   
20250422  1745251200000  11.02  11.06  10.98  11.04   831129  9.159677e+08   
20250423  1745337600000  11.04  11.05  10.97  11.01   586383  6.458424e+08   
20250424  1745424000000  11.00  11.06  10.99  11.03   689256  7.603486e+08   
20250425  1745510400000  11.04  11.05  10.99  11.01   637547  7.023153e+08   

       

In [5]:


# 获取订阅后的行情，并以固定间隔进行刷新,预期会循环打印10次
for i in range(10):
    # 这边做演示，就用for来循环了，实际使用中可以用while True
    kline_data = xtdata.get_market_data_ex([],code_list,period=period)
    print(kline_data)
    time.sleep(3) # 三秒后再次获取行情


{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
Index: []}
{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
Index: []}
{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
Index: []}
{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
Index: []}
{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
Index: []}
{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, settelementPrice, openInterest, preClose, suspendFlag]
Index: []}
{'000001.SZ': Empty DataFrame
Columns: [time, open, high, low, close, volume, amount, se

In [9]:

# 如果不想用固定间隔触发，可以以用订阅后的回调来执行
# 这种模式下当订阅的callback回调函数将会异步的执行，每当订阅的标的tick发生变化更新，callback回调函数就会被调用一次
# 本地已有的数据不会触发callback

# 定义的回测函数
    ## 回调函数中，data是本次触发回调的数据，只有一条
def f(data):
    print(data)

    code_list = list(data.keys())    # 获取到本次触发的标的代码
    print(f'获取到本次触发的标的代码：{code_list}')
    kline_in_callabck = xtdata.get_market_data_ex([],code_list,period = period)    # 在回调中获取klines数据
    print(kline_in_callabck)

for i in code_list:
    xtdata.subscribe_quote(i,period=period,count=10,callback=f) # 订阅时设定回调函数

# 使用回调时，必须要同时使用xtdata.run()来阻塞程序，否则程序运行到最后一行就直接结束退出了。
xtdata.run()





KeyboardInterrupt: 